In [1]:
require("data.table")

Loading required package: data.table



In [2]:
ftirar  <- function( prob, qty )
{
  return(  sum( runif(qty) < prob ) )
}


In [3]:
#variables globales que usan las funciones gimnasio_xxxx
GLOBAL_jugadores  <- c()
GLOBAL_tiros_total  <- 0

In [4]:
#Crea el juego
#a cada jugador se le pone un numero de 1 a 100 en la espalda
#debajo de ese numero esta el indice_de_enceste  que NO puede ser visto por el cazatalentos
gimnasio_init  <- function() 
{
  GLOBAL_jugadores  <<-  sample( c( (501:599 ) / 1000 , 0.7 ) )
  GLOBAL_tiros_total  <<- 0
}


In [5]:
#se le pasa un vector con los IDs de los jugadores y la cantidad de tiros a realizar
#devuelve en un vector cuantos aciertos tuvo cada jugador
gimnasio_tirar  <- function(  pids,  pcantidad )
{
  GLOBAL_tiros_total  <<-  GLOBAL_tiros_total + length( pids )*pcantidad
  res  <- mapply(  ftirar, GLOBAL_jugadores[pids], pcantidad )

  return( res )
}

In [6]:
#El cazatalentos decide a que jugador llevarse
#devuelve la cantidad de tiros libres y si le acerto al verdadero_mejor o no
gimnasio_veredicto  <- function( pid )
{
  return( list("tiros_total"= GLOBAL_tiros_total, 
               "acierto"=     as.integer( GLOBAL_jugadores[pid]==0.7) ))
}

In [7]:
#Estrategia
#En la primera ronda, se hace tirar 50 tiros libres a cada uno de los 100 jugadores ( se gastan 5.000 tiros )
#En la segunda ronda, se hace tirar 50 tiros libres a cada uno de los 100 jugadores (se gastan 5.000 tiros)
#Se eligen a los mejores 10 (sumo los tiros libres encestados en cada ronda y elijo los 10 con mas
#En la tercera ronda, a los 10 mejores de las anteriores rondas se los hace tirar 415 tiros a cada uno
#Se elige el mejor jugador de la tercera ronda

set.seed( 102191 )

In [8]:
Estrategia_A  <- function()
{
    #inicializo el juego
    gimnasio_init()
    #Esta el la planilla del cazatalentos
    planilla_cazatalentos  <- data.table( "id" = 1:100 )
    #Ronda 1  ------------------------------------------------------
    #tiran los 100 jugadores es decir 1:100  50 tiros libres cada uno
    ids_juegan1  <- 1:100   #los jugadores que participan en la ronda,
    
    planilla_cazatalentos[ ids_juegan1,  tiros1 := 50 ]  #registro en la planilla que tiran 90 tiros
    
    #Hago que tiren
    resultado1  <- gimnasio_tirar( ids_juegan1, 50)
    planilla_cazatalentos[ ids_juegan1,  aciertos1 := resultado1 ]  #registro en la planilla
    
    #Ronda 2  ------------------------------------------------------
    #tiran los 100 jugadores es decir 1:100  50 tiros libres cada uno
    ids_juegan2  <- 1:100   #los jugadores que participan en la ronda,

    planilla_cazatalentos[ ids_juegan2,  tiros2 := 50 ]  #registro en la planilla que tiran 90 tiros
    
    #Hago que tiren
    resultado2  <- gimnasio_tirar( ids_juegan2, 50)
    planilla_cazatalentos[ ids_juegan2,  aciertos2 := resultado2 ]  #registro en la planilla
    
    planilla_cazatalentos$aciertos1y2 <- planilla_cazatalentos$aciertos1 + planilla_cazatalentos$aciertos2
    
    #Ronda 3 -------------------------------------------------------
    #los mejores 10 jugadores tiran 415 tiros cada uno
    #aciertos1y2 <- (aciertos1 + aciertos2)
    data_mod <- planilla_cazatalentos[order(planilla_cazatalentos$aciertos1y2, decreasing = TRUE), ] #Orderno de mayor a menor aciertos1y2
    top10  <- data_mod[ , head(.SD, 10) ]
    min  <- min(top10$aciertos1y2)
    ids_juegan3  <- planilla_cazatalentos[ ids_juegan1 ][aciertos1y2 >= min, id ]
    
    planilla_cazatalentos[ ids_juegan3,  tiros3 := 415 ]  #registro en la planilla que tiran 415 tiros
    resultado3  <- gimnasio_tirar( ids_juegan3, 415)
    planilla_cazatalentos[ ids_juegan3,  aciertos3 := resultado3 ]  #registro en la planilla
    
    #El cazatalentos toma una decision, elige al que mas aciertos tuvo en la ronda3
    pos_mejor <-  planilla_cazatalentos[ , which.max(aciertos3) ]
    id_mejor  <-  planilla_cazatalentos[ pos_mejor, id ]
    
    #Finalmente, la hora de la verdadero_mejor
    #Termino el juego
    veredicto  <- gimnasio_veredicto( id_mejor )
    
      return( veredicto )
}

In [9]:
#Aqui hago la Estimacion Montecarlo del porcentaje de aciertos que tiene la estrategia A

tabla_veredictos  <- data.table(  tiros_total=integer(),  acierto=integer() )

In [10]:
for( experimento  in  1:10000 )
{
  if( experimento %% 1000 == 0 )  cat( experimento, " ")  #desprolijo, pero es para saber por donde voy

  veredicto  <- Estrategia_A()
  
  tabla_veredictos  <- rbind( tabla_veredictos, veredicto )
}

1000  2000  3000  4000  5000  6000  7000  8000  9000  10000  

In [11]:
cat("\n")

In [12]:
tiros_total  <-  tabla_veredictos[  , max( tiros_total) ]
tasa_eleccion_correcta  <-  tabla_veredictos[  , mean( acierto) ]

In [13]:
tiros_total
tasa_eleccion_correcta

[1] 19130

[1] 0.9515

In [17]:
# FS: Seteo donde quiero que me baje el csv
setwd("C:/Users/flore/OneDrive/Escritorio/Flor/Maestria/DMEyF")

In [18]:
# FS: Bajo el csv
write.table(tabla_veredictos,"tabla_veredictos en txt.txt")